In [3]:
import os
import tensorflow as tf
import pandas as pd

In [4]:
import numpy as np
import shutil
import matplotlib
import matplotlib.pyplot as plt


In [5]:
pwd

'C:\\Users\\Kevin Tang\\Documents\\HackTheNorth2023\\AI'

In [51]:
SHUFFLE_BUFFER = 500
BATCH_SIZE = 32

In [30]:
DATA_PATH = r"C:\Users\Kevin Tang\Downloads\shuttle.csv"

TEST_PATH = "dataset/test_data.csv"

In [31]:
df = pd.DataFrame([[0, 2, 3], [0, 4, 1], [10, 20, 30]])
df.at[0, 2]

3

In [32]:
df = pd.read_csv(DATA_PATH)
print(df)

       time  rad_flow  fpv_close  fpv_open  high  bypass  bvp_close  bvp_open  \
0        50        21         77         0    28       0         27        48   
1        55         0         92         0     0      26         36        92   
2        53         0         82         0    52      -5         29        30   
3        37         0         76         0    28      18         40        48   
4        37         0         79         0    34     -26         43        46   
...     ...       ...        ...       ...   ...     ...        ...       ...   
43495    46         5         78         0    46       5         32        32   
43496    37         0         79        -1    10       3         43        69   
43497    48         0         78         3    46       0         30        32   
43498    41         0         79         0    38     -25         38        40   
43499    40        -3        100         0    38       0         61        62   

       feature  class  
0  

In [33]:
df = pd.read_csv(DATA_PATH)

df = df.drop('BookingID', axis=1) # axis: 0 for row, 1 for column

print(len(df))

for i in range(len(df)):
    # ARRIVAL YEAR
    df.at[i, "ArrivalYear"] = df.at[i, "ArrivalYear"] % 2000

    # MEAL PARSING
    if df.at[i, "MealPlan"].startswith("Not"):
        df.at[i, "MealPlan"] = 0
    else:
        df.at[i, "MealPlan"] = df.at[i, "MealPlan"].split()[-1]

    # ROOM TYPE
    df.at[i, "RoomType"] = df.at[i, "RoomType"].split()[-1]

    # MARKET SEGMENT
    if df.at[i, "MarketSegment"] == "Online":
        df.at[i, "MarketSegment"] = 5
    elif df.at[i, "MarketSegment"] == "Offline":
        df.at[i, "MarketSegment"] = 4
    elif df.at[i, "MarketSegment"] == "Corporate":
        df.at[i, "MarketSegment"] = 3
    elif df.at[i, "MarketSegment"] == "Complementary":
        df.at[i, "MarketSegment"] = 2
    elif df.at[i, "MarketSegment"] == "Aviation":
        df.at[i, "MarketSegment"] = 1
    else:
        print("Recheck data.")

    # BOOKING STATUS
    if df.at[i, "BookingStatus"] == "Canceled":
        df.at[i, "BookingStatus"] = 0
    elif df.at[i, "BookingStatus"] == "Not_Canceled":
        df.at[i, "BookingStatus"] = 1

# BOOKING STATUS
target = df.pop("BookingStatus")
df

KeyError: "['BookingID'] not found in axis"

# Normalizing the dataframe

In [107]:
# REMOVED_COL = "BookingID"
PREDICT_COL = "class"
ENCODING = []

df = pd.read_csv(DATA_PATH)
# df = df.drop(REMOVED_COL, axis=1) # axis: 0 for row, 1 for column


for col in df.columns:
    if not all(isinstance(x, (int, float)) for x in df[col]):
        lst = np.unique(df[col].astype(str))
        if col == PREDICT_COL:
            for i in range(len(lst)):
                ENCODING.append((i, lst[i]))
        for i in range(len(lst)):
            df[col] = df[col].replace(lst[i], i)
        lst = []


target = df.pop(PREDICT_COL)
target = tf.one_hot(target, depth=9)
print(target)
print(ENCODING)
df
print(len(ENCODING))

tf.Tensor(
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]], shape=(43500, 9), dtype=float32)
[]
0


In [36]:
# Name: BookingStatus, Length: 29020, dtype: object
# dtype can't be object or will get error for normalizer(numeric_features.iloc[:3])
target = target.astype('int')
df = df.astype('int')
print(type(target))
target

<class 'pandas.core.series.Series'>


0        2
1        4
2        1
3        1
4        1
        ..
43495    1
43496    1
43497    1
43498    1
43499    1
Name: class, Length: 43500, dtype: int32

In [37]:
numeric_feature_names = [name for name in df.columns]
# numeric_feature_names = ['LeadTime', 'ArrivalYear', 'ArrivalMonth',  'ArrivalDate', 'NumWeekendNights', "NumWeekNights", "MealPlan", "Parking", "RoomType", "NumAdults", "NumChildren", "MarketSegment", "RepeatedGuest", "NumPrevCancellations", "NumPreviousNonCancelled", "AvgRoomPrice", "SpecialRequests"]
numeric_features = df[numeric_feature_names]
numeric_features.head()

print(type(numeric_features))

<class 'pandas.core.frame.DataFrame'>


In [38]:
# numeric_features = np.asarray(numeric_features).astype('float32')

tf.convert_to_tensor(numeric_features)


<tf.Tensor: shape=(43500, 9), dtype=int32, numpy=
array([[ 50,  21,  77, ...,  27,  48,  22],
       [ 55,   0,  92, ...,  36,  92,  56],
       [ 53,   0,  82, ...,  29,  30,   2],
       ...,
       [ 48,   0,  78, ...,  30,  32,   2],
       [ 41,   0,  79, ...,  38,  40,   2],
       [ 40,  -3, 100, ...,  61,  62,   2]])>

In [54]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features.to_numpy())


In [40]:
numeric_features = pd.DataFrame(numeric_features)
type(numeric_features)
print(numeric_features)

       time  rad_flow  fpv_close  fpv_open  high  bypass  bvp_close  bvp_open  \
0        50        21         77         0    28       0         27        48   
1        55         0         92         0     0      26         36        92   
2        53         0         82         0    52      -5         29        30   
3        37         0         76         0    28      18         40        48   
4        37         0         79         0    34     -26         43        46   
...     ...       ...        ...       ...   ...     ...        ...       ...   
43495    46         5         78         0    46       5         32        32   
43496    37         0         79        -1    10       3         43        69   
43497    48         0         78         3    46       0         30        32   
43498    41         0         79         0    38     -25         38        40   
43499    40        -3        100         0    38       0         61        62   

       feature  
0         

In [41]:
normalizer(numeric_features.iloc[:3])


<tf.Tensor: shape=(3, 9), dtype=float32, numpy=
array([[ 0.14285067,  0.27136692, -0.9363581 , -0.00640761, -0.3008231 ,
        -0.00723336, -0.7669769 , -0.13511065,  0.31329548],
       [ 0.5509323 ,  0.00262505,  0.7474303 , -0.00640761, -1.5909599 ,
         0.13762584, -0.08180439,  1.9149319 ,  1.6389323 ],
       [ 0.38769963,  0.00262505, -0.37509528, -0.00640761,  0.8050085 ,
        -0.03509089, -0.61471635, -0.9737644 , -0.46649086]],
      dtype=float32)>

In [42]:
print(normalizer)
print(normalizer.input)
print(normalizer.output)


AttributeError: Layer normalization_1 is not connected, no input to return.

In [43]:
len(numeric_features.iloc[0])

9

In [44]:
from math import floor
# some complex math
nodes = len(numeric_features)/(4 * (len(numeric_features.iloc[0]) + len(ENCODING)))
res = 0
for i in range(floor(nodes), 0, -1):
    if (i & (i - 1)) == 0:
        res = i
        break

lst = [res]
for i in range(3):
    res /= 2
    lst.append(res)
print(lst)


[1024, 512.0, 256.0, 128.0]


In [45]:
from math import floor

# some complex math
nodes = len(numeric_features)/(2 * (len(numeric_features.iloc[0]) + len(ENCODING)))
res = floor(nodes/15)

print(res*1, res*2, res*4, res*8)


# for i in range(nodes, 0, -1):
#     if (i & (i - 1)) == 0:
#         res = i
#         break
#
# lst = [res]
# for i in range(4):
#     res /= 2
#     lst.append(res)
# print(lst)

161 322 644 1288


In [104]:
def get_basic_model():
    model = tf.keras.Sequential([
        normalizer,
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(9, activation='sigmoid')
        ])

    model.compile(optimizer='adam',
                    loss="categorical_crossentropy", # tf.keras.losses.BinaryCrossentropy(from_logits=True)
                    metrics=['accuracy'])
    return model

model = get_basic_model()

0

In [56]:
print(numeric_features.iloc[:3])

   time  rad_flow  fpv_close  fpv_open  high  bypass  bvp_close  bvp_open  \
0    50        21         77         0    28       0         27        48   
1    55         0         92         0     0      26         36        92   
2    53         0         82         0    52      -5         29        30   

   feature  
0       22  
1       56  
2        2  


In [76]:
target

0        2
1        4
2        1
3        1
4        1
        ..
43495    1
43496    1
43497    1
43498    1
43499    1
Name: class, Length: 43500, dtype: int32

In [86]:
numeric_features.shape, target.shape

((43500, 9), TensorShape([43500, 9]))

In [102]:
model.predict(numeric_features).shape

1360/1360 [==============================] - 1s 521us/step


(43500, 9)

In [105]:
model.fit(numeric_features, target)

1360/1360 [==============================] - 2s 833us/step - loss: 0.2303 - accuracy: 0.9441


In [106]:
model = get_basic_model()
print(numeric_features.shape)
print(target.shape)
print(target)
history = model.fit(numeric_features, target, epochs=30, batch_size=BATCH_SIZE)


(43500, 9)
(43500, 9)
tf.Tensor(
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]], shape=(43500, 9), dtype=float32)
Epoch 1/30
1360/1360 [==============================] - 2s 817us/step - loss: 0.2337 - accuracy: 0.9417
Epoch 2/30
1360/1360 [==============================] - 1s 785us/step - loss: 0.0775 - accuracy: 0.9881
Epoch 3/30
1360/1360 [==============================] - 1s 785us/step - loss: 0.0548 - accuracy: 0.9901
Epoch 4/30
1360/1360 [==============================] - 1s 792us/step - loss: 0.0449 - accuracy: 0.9926
Epoch 5/30
1360/1360 [==============================] - 1s 788us/step - loss: 0.0335 - accuracy: 0.9934
Epoch 6/30
1360/1360 [==============================] - 1s 790us/step - loss: 0.0263 - accuracy: 0.9936
Epoch 7/30
1360/1360 [==============================] - 1s 782us/step - loss: 0.0248 - accuracy: 0.9941
Epoch 8/30
1360/1360 [===========================

In [45]:
numeric_dataset = tf.data.Dataset.from_tensor_slices((numeric_features, target))

for row in numeric_dataset.take(3):
  print(row)


(<tf.Tensor: shape=(17,), dtype=int32, numpy=
array([  10, 2018,    3,   31,    0,    1,    0,    0,    0,    1,    0,
          2,    0,    0,    0,   95,    0])>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(17,), dtype=int32, numpy=
array([ 116, 2018,    2,   28,    2,    1,    0,    0,    0,    1,    0,
          4,    0,    0,    0,   61,    0])>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(17,), dtype=int32, numpy=
array([  11, 2018,    7,   25,    1,    2,    0,    0,    0,    2,    1,
          4,    0,    0,    0,  129,    1])>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)


In [46]:
numeric_batches = numeric_dataset.shuffle(1000).batch(BATCH_SIZE)

model = get_basic_model()
history = model.fit(numeric_batches, epochs=90)

Epoch 1/90
29/29 [==============================] - 1s 3ms/step - loss: 0.5702 - accuracy: 0.6997
Epoch 2/90
29/29 [==============================] - 0s 3ms/step - loss: 0.4738 - accuracy: 0.7826
Epoch 3/90
29/29 [==============================] - 0s 3ms/step - loss: 0.4436 - accuracy: 0.8005
Epoch 4/90
29/29 [==============================] - 0s 3ms/step - loss: 0.4265 - accuracy: 0.8103
Epoch 5/90
29/29 [==============================] - 0s 2ms/step - loss: 0.4111 - accuracy: 0.8187
Epoch 6/90
29/29 [==============================] - 0s 2ms/step - loss: 0.4040 - accuracy: 0.8251
Epoch 7/90
29/29 [==============================] - 0s 3ms/step - loss: 0.3958 - accuracy: 0.8281
Epoch 8/90
29/29 [==============================] - 0s 2ms/step - loss: 0.3841 - accuracy: 0.8326
Epoch 9/90
29/29 [==============================] - 0s 3ms/step - loss: 0.3770 - accuracy: 0.8369
Epoch 10/90
29/29 [==============================] - 0s 3ms/step - loss: 0.3723 - accuracy: 0.8387
Epoch 11/90
29/29 [

In [243]:
plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

# 💀💀💀💀💀💀💀💀💀

NameError: name 'plt' is not defined

In [47]:
# pd.DataFrame(numeric_features.numpy())
print(type(numeric_features))
print(numeric_features)
predictions = model.predict(np.array(numeric_features.iloc[:20]))

print("asdf")

lst = ["Canceled",
"Canceled",
"Not_Canceled",
"Not_Canceled",
"Not_Canceled",
"Canceled",
"Not_Canceled",
"Not_Canceled",
"Canceled",
"Not_Canceled",
"Not_Canceled",
"Not_Canceled",
"Canceled",
"Canceled",
"Not_Canceled",
"Not_Canceled",
"Not_Canceled",
"Not_Canceled",
"Canceled"]
# print(predictions)

for i in range(len(predictions)-1):
    print(f"{predictions[i]}: {lst[i]}")
    if predictions[i][0] > predictions[i][1]:
        print("Predicted: cancled")
    else:
        print("Predicted: not_canceled")


<class 'pandas.core.frame.DataFrame'>
       LeadTime  ArrivalYear  ArrivalMonth  ArrivalDate  NumWeekendNights  \
0            10         2018             3           31                 0   
1           116         2018             2           28                 2   
2            11         2018             7           25                 1   
3             3         2017             9           12                 0   
4            28         2018             3            7                 1   
...         ...          ...           ...          ...               ...   
29015         2         2018            11           21                 1   
29016        18         2018             4           12                 0   
29017         8         2017             9           29                 0   
29018        20         2018             9           18                 2   
29019       274         2018             7           31                 2   

       NumWeekNights  MealPlan  Parki

In [11]:
pwd

'C:\\Users\\Kevin Tang\\Documents\\HackTheNorth2023\\AI'

In [63]:
DATA_PATH = "dataset/updated_test_data.csv"

# prediction
REMOVED_COL = "BookingID"
PREDICT_COL = "BookingStatus"

df = pd.read_csv(DATA_PATH)
df = df.drop(REMOVED_COL, axis=1) # axis: 0 for row, 1 for column
# target = df.drop(PREDICT_COL)

for col in df.columns:
    if not all(isinstance(x, (int, float)) for x in df[col]):
        lst = np.unique(df[col].astype(str))
        for i in range(len(lst)):
            df[col] = df[col].replace(lst[i], i)


# target = df.drop(PREDICT_COL)
print(len(numeric_feature_names))
df = df.astype('int')
numeric_feature_names = [name for name in df.columns]
print(len(numeric_feature_names))

numeric_features = df[numeric_feature_names]
numeric_features.head()
tf.convert_to_tensor(numeric_features)
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features.to_numpy())


# new_model = tf.keras.models.load_model('../backend/local/UAQ3GzHi8TNmOS9qjtnEroOcIuy2/model_ac699aad-290c-47bc-8f7c-a80eb84b0b01/model.keras')

new_model = tf.keras.models.load_model('../backend/local/UAQ3GzHi8TNmOS9qjtnEroOcIuy2/model_ac699aad-290c-47bc-8f7c-a80eb84b0b01/model')

# new_model.load_weights('../backend/local/UAQ3GzHi8TNmOS9qjtnEroOcIuy2/model_ac699aad-290c-47bc-8f7c-a80eb84b0b01/model')

predictions = new_model.predict(np.array(numeric_features))



17
17
227/227 [==============================] - 0s 549us/step


In [58]:
print(predictions)

[[0.15406722 0.9829524 ]
 [0.16708525 0.9810974 ]
 [0.02197331 0.99994904]
 ...
 [0.0456804  0.99965805]
 [0.07709794 0.99846077]
 [0.09470014 0.9967107 ]]
